In [38]:
# Import the Pandas library
import pandas as pd

In [2]:
# Create a reference the CSV file desired
csv_path1 = "Resources/schools_complete.csv"
csv_path2 = "Resources/students_complete.csv"

# Read the CSV into a Pandas DataFrame
district_df = pd.read_csv(csv_path1)
students_df = pd.read_csv(csv_path2)

district_df=district_df.rename(columns={"name":"school_name"})
students_df=students_df.rename(columns={"school":"school_name"})
# Print the first five rows of data to the screen
district_df.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [3]:
students_df.head()

,Student ID,name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [4]:
schools = district_df["school_name"].unique()
print (schools)

['Huang High School' 'Figueroa High School' 'Shelton High School'
 'Hernandez High School' 'Griffin High School' 'Wilson High School'
 'Cabrera High School' 'Bailey High School' 'Holden High School'
 'Pena High School' 'Wright High School' 'Rodriguez High School'
 'Johnson High School' 'Ford High School' 'Thomas High School']


In [5]:
students_df.describe()

,Student ID,reading_score,math_score
count,39170.000000,39170.00000,39170.000000
mean,19584.500000,81.87784,78.985371
std,11307.549359,10.23958,12.309968
min,0.000000,63.00000,55.000000
25%,9792.250000,73.00000,69.000000
50%,19584.500000,82.00000,79.000000
75%,29376.750000,91.00000,89.000000
max,39169.000000,99.00000,99.000000


In [6]:
# 
num_schools=district_df["School ID"].count()
#num_schools=students_df['school_name'].value_counts()
num_students=district_df["size"].sum()
total_budget=district_df["budget"].sum()
num_schools, num_students, total_budget

(15, 39170, 24649428)

In [7]:
# Convert the state_counts Series into a DataFrame
avg_reading_score = students_df["reading_score"].mean()
avg_math_score = students_df["math_score"].mean()
avg_reading_score, avg_math_score

(81.87784018381414, 78.98537145774827)

In [8]:
# Establish pass/fail bins for reading and math scores
bins = [0, 70, 100]
bins_lables = ["Fail", "Pass"]
students_df['math_status']=pd.cut(students_df['math_score'], bins, labels=bins_lables)
pf_counts_math=students_df['math_status'].value_counts()
# Do the same for reading scores
students_df['reading_status']=pd.cut(students_df['reading_score'], bins, labels=bins_lables)
pf_counts_reading=students_df['reading_status'].value_counts()
pf_counts_math, pf_counts_reading

(Pass    28356
 Fail    10814
 Name: math_status, dtype: int64, Pass    32500
 Fail     6670
 Name: reading_status, dtype: int64)

In [9]:
fail_count_math=pf_counts_math.iloc[1]
pass_count_math=pf_counts_math.iloc[0]
fail_count_reading=pf_counts_reading.iloc[1]
pass_count_reading=pf_counts_reading.iloc[0]
pass_rate_math=pass_count_math/num_students*100
pass_rate_reading=pass_count_reading/num_students*100
#
fail_count_math, pass_count_math, fail_count_reading, pass_count_reading, pass_rate_math, pass_rate_reading

(10814, 28356, 6670, 32500, 72.392136839417915, 82.971661986213945)

In [10]:
district_summary=pd.DataFrame({"Total Schools":[num_schools],
                               "Total Students":[num_students],
                               "Total Budget":[total_budget],
                               "Average Math Score":[avg_math_score],
                               "Average Reading Score":[avg_reading_score],
                               "% Passing Math":[pass_rate_math],
                               "% Passing Reading":[pass_rate_reading],
                               "% Overall Passing Rate":[(pass_rate_math+pass_rate_reading)/2]
                               })
district_summary=district_summary[['Total Schools','Total Students','Total Budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']]
district_summary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,72.392137,82.971662,77.681899


In [11]:
school_summary=district_df[['school_name','type','size','budget']]
#school_summary=district_df[['type','size','budget']]
school_summary['Budget per Stdnt']=district_df['budget']/district_df['size']
school_group=students_df.groupby("school_name")
math_avg = school_group['math_score'].sum()/school_group['math_score'].count()
reading_avg = school_group['reading_score'].sum()/school_group['reading_score'].count()
school_summary['Average math score']=math_avg.values
school_summary['Average reading score']=reading_avg.values
#school_summary.index = school_summary.school_name
print (school_summary)

              school_name      type  size   budget  Budget per Stdnt  \
0       Huang High School  District  2917  1910635             655.0   
1    Figueroa High School  District  2949  1884411             639.0   
2     Shelton High School   Charter  1761  1056600             600.0   
3   Hernandez High School  District  4635  3022020             652.0   
4     Griffin High School   Charter  1468   917500             625.0   
5      Wilson High School   Charter  2283  1319574             578.0   
6     Cabrera High School   Charter  1858  1081356             582.0   
7      Bailey High School  District  4976  3124928             628.0   
8      Holden High School   Charter   427   248087             581.0   
9        Pena High School   Charter   962   585858             609.0   
10     Wright High School   Charter  1800  1049400             583.0   
11  Rodriguez High School  District  3999  2547363             637.0   
12    Johnson High School  District  4761  3094650             6

In [12]:
print (school_summary.index)

RangeIndex(start=0, stop=15, step=1)


In [13]:
#school_pf_counts_reading=school_group['Reading Status'].value_counts(normalize=True)
#school_pf_counts_math=school_group['Math Status'].value_counts(normalize=True).loc[(school_group['Math Status']=='Pass')]
school_pf_counts_math=school_group['math_status'].value_counts(normalize=True)
school_pf_counts_reading=school_group['reading_status'].value_counts(normalize=True)
print (school_pf_counts_math, school_pf_counts_reading)
#school_summary['% pass math']=school_pf_counts_math['school_name']
#school_prate_math=school_pf_counts_math
#school_summary['% Passing Math']=school_pf_counts_math.values
#school_summary['% Passing Reading']=school_group['Reading Status'].value_counts(normalize=True).iloc[0]*100
#print (school_pf_counts_math)
#school_summary['Overall Passing Rate']=(school_summary['% Passing Math']+school_summary['% Passing Reading'])/2
#print (school_summary)
#school_prate_math=school_group.loc[(school_group['Math Status'] == 'Pass')]
#print (school_pf_counts_math.loc[(school_group["Math Status"]=='Pass')])

school_name            math_status
Bailey High School     Pass           0.646302
                       Fail           0.353698
Cabrera High School    Pass           0.895587
                       Fail           0.104413
Figueroa High School   Pass           0.637504
                       Fail           0.362496
Ford High School       Pass           0.657539
                       Fail           0.342461
Griffin High School    Pass           0.897139
                       Fail           0.102861
Hernandez High School  Pass           0.647465
                       Fail           0.352535
Holden High School     Pass           0.906323
                       Fail           0.093677
Huang High School      Pass           0.633185
                       Fail           0.366815
Johnson High School    Pass           0.638521
                       Fail           0.361479
Pena High School       Pass           0.916840
                       Fail           0.083160
Rodriguez High School  Pa

In [14]:
print (type(school_pf_counts_math))
#school_group=school_group.rename(columns={"Math Status":"math_status"})

<class 'pandas.core.series.Series'>


In [15]:
school_prate_math = []
school_prate_reading = []
school_prate_overall = []
for i in range(0,30,2):
    school_prate_math.append(school_pf_counts_math.iloc[i]*100)
    school_prate_reading.append(school_pf_counts_reading.iloc[i]*100)
    school_prate_overall.append((school_pf_counts_reading.iloc[i]*100 + school_pf_counts_math.iloc[i]*100) / 2)
print (school_prate_overall)
#print (school_pf_counts_math.loc['school_name' == 'Cabrera High School'])
#school_prate_math=school_pf_counts_math[math_status]

[71.965434083601281, 91.711517761033377, 71.091895557816201, 71.631982475355983, 91.553133514986371, 71.4670981661273, 91.686182669789218, 71.066163866986628, 71.067002730518794, 91.943866943866936, 70.905226306576651, 91.254968767745595, 91.559633027522935, 92.093736311870344, 91.861111111111114]


In [16]:
school_summary['% Passing Math'] = school_prate_math
school_summary['% Passing Reading'] = school_prate_reading
school_summary['% Overall Passing'] = school_prate_overall
school_summary.set_index(schools, drop=True, inplace=True)
# Remove the school_name column since it's the new index
school_summary = school_summary [['type', 'size', 'budget', 'Budget per Stdnt', 'Average math score', 'Average reading score',
                                  '% Passing Math', '% Passing Reading', '% Overall Passing']]
school_summary.head(15)

,type,size,budget,Budget per Stdnt,Average math score,Average reading score,% Passing Math,% Passing Reading,% Overall Passing
Huang High School,District,2917,1910635,655.0,77.048432,81.033963,64.630225,79.300643,71.965434
Figueroa High School,District,2949,1884411,639.0,83.061895,83.975780,89.558665,93.864370,91.711518
Shelton High School,Charter,1761,1056600,600.0,76.711767,81.158020,63.750424,78.433367,71.091896
Hernandez High School,District,4635,3022020,652.0,77.102592,80.746258,65.753925,77.510040,71.631982
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,89.713896,93.392371,91.553134
Wilson High School,Charter,2283,1319574,578.0,77.289752,80.934412,64.746494,78.187702,71.467098
Cabrera High School,Charter,1858,1081356,582.0,83.803279,83.814988,90.632319,92.740047,91.686183
Bailey High School,District,4976,3124928,628.0,76.629414,81.182722,63.318478,78.813850,71.066164
Holden High School,Charter,427,248087,581.0,77.072464,80.966394,63.852132,78.281874,71.067003
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,91.683992,92.203742,91.943867


In [17]:
# Generate list of the top 5 schools based on Overall Passing Rate
school_summary.sort_values(by="% Overall Passing", ascending=False).head()

,type,size,budget,Budget per Stdnt,Average math score,Average reading score,% Passing Math,% Passing Reading,% Overall Passing
Ford High School,District,2739,1763916,644.0,83.274201,83.989488,90.932983,93.254490,92.093736
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,91.683992,92.203742,91.943867
Thomas High School,Charter,1635,1043130,638.0,83.682222,83.955000,90.277778,93.444444,91.861111
Figueroa High School,District,2949,1884411,639.0,83.061895,83.975780,89.558665,93.864370,91.711518
Cabrera High School,Charter,1858,1081356,582.0,83.803279,83.814988,90.632319,92.740047,91.686183


In [18]:
# Generate list of the bottom 5 schools based on Overall Passing Rate
school_summary.sort_values(by="% Overall Passing").head()

,type,size,budget,Budget per Stdnt,Average math score,Average reading score,% Passing Math,% Passing Reading,% Overall Passing
Wright High School,Charter,1800,1049400,583.0,76.842711,80.744686,64.066017,77.744436,70.905226
Bailey High School,District,4976,3124928,628.0,76.629414,81.182722,63.318478,78.813850,71.066164
Holden High School,Charter,427,248087,581.0,77.072464,80.966394,63.852132,78.281874,71.067003
Shelton High School,Charter,1761,1056600,600.0,76.711767,81.158020,63.750424,78.433367,71.091896
Wilson High School,Charter,2283,1319574,578.0,77.289752,80.934412,64.746494,78.187702,71.467098


In [19]:
grade_group=students_df.groupby(["school_name","grade"])
# school_summary = pd.DataFrame([school_group.school.unique())
#pd.DataFrame(grade_group.size().reset_index(name = "foo"))
#grade_group.mean()
grade_grp_df = pd.DataFrame(grade_group.mean())
grade_grp_df = grade_grp_df [['reading_score','math_score']]
grade_grp_df.head(8)

reading_score  math_score
school_name         grade                           
Bailey High School  10th       80.907183   76.996772
                    11th       80.945643   77.515588
                    12th       80.912451   76.492218
                    9th        81.303155   77.083676
Cabrera High School 10th       84.253219   83.154506
                    11th       83.788382   82.765560
                    12th       84.287958   83.277487
                    9th        83.676136   83.094697

In [20]:
grade_grp_9th = students_df[students_df['grade'] == '9th']
grade_grp_10th = students_df[students_df['grade'] == '10th']
school_grp_9th = grade_grp_9th.groupby(['school_name']).mean()['math_score']
school_grp_10th = grade_grp_10th.groupby(['school_name']).mean()['math_score']
MathScoresByGrade = pd.DataFrame({'9th' : school_grp_9th,
                                  '10th' : school_grp_10th})
MathScoresByGrade

,10th,9th
school_name,,
Bailey High School,76.996772,77.083676
Cabrera High School,83.154506,83.094697
Figueroa High School,76.539974,76.403037
Ford High School,77.672316,77.361345
Griffin High School,84.229064,82.044010
Hernandez High School,77.337408,77.438495
Holden High School,83.429825,83.787402
Huang High School,75.908735,77.027251
Johnson High School,76.691117,77.187857


In [21]:
#solution = pd.DataFrame()
#for i in students_df['grade'].unique():
grade_grp_9th = students_df[students_df['grade'] == '9th']
school_grp_9th = grade_grp_9th.groupby(['school_name']).mean()['math_score']
solution = pd.DataFrame({'9th' : school_grp_9th})
for i in '10th', '11th', '12th':
    i = pd.DataFrame({i : students_df[students_df['grade'] == i].groupby(['school_name']).mean()['math_score']})
    solution = solution.join(i)
solution.head(15)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [22]:
grade_grp_9th = students_df[students_df['grade'] == '9th']
school_grp_9th = grade_grp_9th.groupby(['school_name']).mean()['reading_score']
solution = pd.DataFrame({'9th' : school_grp_9th})
for i in '10th', '11th', '12th':
    i = pd.DataFrame({i : students_df[students_df['grade'] == i].groupby(['school_name']).mean()['reading_score']})
    solution = solution.join(i)
solution.head(15)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [24]:
# The code below send jupyter into a tight loop and not responsive (memory usage 97+, CPU in the teens)
# Nothing gets pritned to the screen!!
#grade_groups = []
#for i in students_df['grade'].unique():
#    grade_groups.append(students_df[students_df['grade'] == i])
#    print (i)
#print (grade_groups[0])

In [25]:
print (school_summary['Budget per Stdnt'].min(), school_summary['Budget per Stdnt'].max())

578.0 655.0


In [26]:
spending_cats = [0, 595, 615, 635, 660]
school_summary['spending_cat']=pd.cut(school_summary['Budget per Stdnt'], spending_cats)
reduced_school_summary = school_summary [['spending_cat','Average math score', 'Average reading score', '% Passing Math',
                                          '% Passing Reading', '% Overall Passing']]
scores_by_spending = reduced_school_summary.groupby('spending_cat')
scores_by_spending.mean()

,Average math score,Average reading score,% Passing Math,% Passing Reading,% Overall Passing
spending_cat,,,,,
"(0, 595]",78.752051,81.615120,70.824240,81.738515,76.281377
"(595, 615]",80.275842,82.601359,77.717208,85.318555,81.517881
"(615, 635]",79.990456,82.499740,76.516187,86.103110,81.309649
"(635, 660]",81.563878,83.039306,83.037107,88.985288,86.011198


In [27]:
print (school_summary['size'].min(), school_summary['size'].max())

427 4976


In [37]:
size_cats = [0, 1500, 3000, 5000]
size_labels = ['Small (<1500)', 'Medium (1500-3000)', 'Large (3000-5000)']
school_summary['size_cat']=pd.cut(school_summary['size'], size_cats, labels=size_labels)
reduced_school_summary = school_summary [['size_cat','Average math score', 'Average reading score', '% Passing Math',
                                          '% Passing Reading', '% Overall Passing']]
scores_by_size = reduced_school_summary.groupby('size_cat')
scores_by_size.mean().head()

,Average math score,Average reading score,% Passing Math,% Passing Reading,% Overall Passing
size_cat,,,,,
Small (<1500),81.421293,82.942617,81.750007,87.959329,84.854668
Medium (1500-3000),80.214282,82.450792,77.324363,85.871187,81.597775
Large (3000-5000),80.127452,82.375908,77.294644,85.461730,81.378187


In [35]:
reduced_school_summary2 = school_summary [['type','Average math score', 'Average reading score', '% Passing Math',
                                          '% Passing Reading', '% Overall Passing']]
#print(reduced_school_summary.head(5))
scores_by_type = reduced_school_summary2.groupby('type')
scores_by_type.mean().head()

,Average math score,Average reading score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,80.324201,82.429369,77.340381,85.553498,81.446940
District,80.556334,82.643266,79.185779,86.895203,83.040491


In [29]:
#School performance broken down by average spending ranges
#students_df['math_status']=pd.cut(students_df['math_score'], bins, labels=bins_lables)
#print (type(school_summary), school_summary.index, school_summary.columns)
spending_cats = [0, 595, 615, 635, 655]
#spending_range_labels = ['<595', '"Pass"']
school_summary['spending_cat']=pd.cut(school_summary['Budget per Stdnt'], spending_cats)
scores_by_spending = school_summary.set_index('spending_cat')
scores_by_spending = scores_by_spending [['Budget per Stdnt', 'Average math score', 'Average reading score', '% Passing Math', 
                                          '% Passing Reading','% Overall Passing']]
#scores_by_spending.set_index('spending_cat', inplace=True)
scores_by_spending.head(15)
#per_student_spending_df = per_student_spending_df [['Average math score', 'Average reading score',
#                                  '% Passing Math', '% Passing Reading', '% Overall Passing']]

#per_student_spending_df.head()

,Budget per Stdnt,Average math score,Average reading score,% Passing Math,% Passing Reading,% Overall Passing
spending_cat,,,,,,
"(635, 655]",655.0,77.048432,81.033963,64.630225,79.300643,71.965434
"(635, 655]",639.0,83.061895,83.975780,89.558665,93.864370,91.711518
"(595, 615]",600.0,76.711767,81.158020,63.750424,78.433367,71.091896
"(635, 655]",652.0,77.102592,80.746258,65.753925,77.510040,71.631982
"(615, 635]",625.0,83.351499,83.816757,89.713896,93.392371,91.553134
"(0, 595]",578.0,77.289752,80.934412,64.746494,78.187702,71.467098
"(0, 595]",582.0,83.803279,83.814988,90.632319,92.740047,91.686183
"(615, 635]",628.0,76.629414,81.182722,63.318478,78.813850,71.066164
"(0, 595]",581.0,77.072464,80.966394,63.852132,78.281874,71.067003


In [30]:
size_cats = [0, 1500, 3000, 3500]
size_labels = ['Small', 'Medium', 'Large']
school_summary['size_cat']=pd.cut(school_summary['size'], size_cats, labels=size_labels)
scores_by_size = school_summary.set_index('size_cat')
scores_by_size = scores_by_size [[ 'Average math score', 'Average reading score', '% Passing Math', 
                                          '% Passing Reading','% Overall Passing']]
school_summary.head()

,type,size,budget,Budget per Stdnt,Average math score,Average reading score,% Passing Math,% Passing Reading,% Overall Passing,spending_cat,size_cat
Huang High School,District,2917,1910635,655.0,77.048432,81.033963,64.630225,79.300643,71.965434,"(635, 655]",Medium
Figueroa High School,District,2949,1884411,639.0,83.061895,83.975780,89.558665,93.864370,91.711518,"(635, 655]",Medium
Shelton High School,Charter,1761,1056600,600.0,76.711767,81.158020,63.750424,78.433367,71.091896,"(595, 615]",Medium
Hernandez High School,District,4635,3022020,652.0,77.102592,80.746258,65.753925,77.510040,71.631982,"(635, 655]",NaN
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,89.713896,93.392371,91.553134,"(615, 635]",Small
